# Deep Conditional Density Estimation

In this notebook, we explore the use of Deep Gaussian processes and Latent Variables to model a dataset with heteroscedastic noise.

In [ ]:
import tensorflow as tf
import gpflow
import gpflux
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow_probability as tfp
from sklearn.neighbors import KernelDensity


tf.keras.backend.set_floatx("float64")

## Load data

In [ ]:
def motorcycle_data():
    """ Return inputs and outputs for the motorcycle dataset. We normalise the outputs. """
    import pandas as pd
    df = pd.read_csv("./data/motor.csv", index_col=0)
    X, Y = df["times"].values.reshape(-1, 1), df["accel"].values.reshape(-1, 1)
    Y = (Y - Y.mean()) / Y.std()
    X /= X.max()
    return X, Y

In [ ]:
X, Y = motorcycle_data()
num_data, d_xim = X.shape

X_MARGIN, Y_MARGIN = 0.1, 0.5
fig, ax = plt.subplots()
ax.scatter(X, Y, marker='x', color='k');
ax.set_ylim(Y.min() - Y_MARGIN, Y.max() + Y_MARGIN);
ax.set_xlim(X.min() - X_MARGIN, X.max() + X_MARGIN);

## Standard single layer Sparse Variational GP

We first show that a single layer SVGP performs quite poorly on this dataset. In the following code block we define the kernel, inducing variable, GP layer and likelihood of our shallow GP:

In [ ]:
NUM_INDUCING = 20

kernel = gpflow.kernels.SquaredExponential()
inducing_variable = gpflow.inducing_variables.InducingPoints(
    np.linspace(X.min(), X.max(), NUM_INDUCING).reshape(-1, 1)
)
gp_layer = gpflux.layers.GPLayer(
    kernel, inducing_variable, num_data=num_data, num_latent_gps=1
)
likelihood_layer = gpflux.layers.LikelihoodLayer(gpflow.likelihoods.Gaussian(0.1))



We can now encapsulate `gp_layer` in a GPflux DeepGP model:

In [ ]:

single_layer_dgp = gpflux.models.DeepGP([gp_layer], likelihood_layer)
model = single_layer_dgp.as_training_model()
model.compile(tf.optimizers.Adam(0.01))

history = model.fit({"inputs": X, "targets": Y}, epochs=int(1e3), verbose=0)
fig, ax = plt.subplots()
ax.plot(history.history["loss"])
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')

In [ ]:
fig, ax = plt.subplots()
num_data_test = 200
X_test = np.linspace(X.min() - X_MARGIN, X.max() + X_MARGIN, num_data_test).reshape(-1, 1)
model = single_layer_dgp.as_prediction_model()
out = model(X_test)

mu = out.y_mean.numpy().squeeze()
var = out.y_var.numpy().squeeze()
X_test = X_test.squeeze()

for i in [1, 2]:
    lower = mu - i * np.sqrt(var)
    upper = mu + i * np.sqrt(var)
    ax.fill_between(X_test, lower, upper, color="C1", alpha=0.3)

ax.set_ylim(Y.min() - Y_MARGIN, Y.max() + Y_MARGIN)
ax.set_xlim(X.min() - X_MARGIN, X.max() + X_MARGIN)
ax.plot(X, Y, "kx", alpha=0.5)
ax.plot(X_test, mu, "C1")
ax.set_xlabel('time')
ax.set_ylabel('acc')


The errorbars of the single layer model are not good.

## Deep Gaussian process with latent variables

To tackle the problem we suggest a Deep Gaussian process with a latent variable in the first layer. The latent variable will be able to capture the heteroskedasticity, while the two layered deep GP is able to model the sharp transitions.

### Latent Variable Layer

This layer concatenates the inputs with a latent variable. See Dutordoir, Salimbeni et al. Conditional Density with Gaussian processes (2018) for full details.

In [ ]:
w_dim = 1
prior_means = np.zeros(w_dim)
prior_std = np.ones(w_dim)
encoder = gpflux.encoders.DirectlyParameterizedNormalDiag(num_data, w_dim)
prior = tfp.distributions.MultivariateNormalDiag(prior_means, prior_std)
lv = gpflux.layers.LatentVariableLayer(prior, encoder)

### First GP layer

GP Layer with two dimensional input because it acts on the inputs and the one-dimensional latent variable. We use a Squared Exponential kernel, a zero mean function, and inducing points, whose pseudo input locations are carefully chosen.

In [ ]:

kernel = gpflow.kernels.SquaredExponential(lengthscales=[.05, .2], variance=1.)
inducing_variable = gpflow.inducing_variables.InducingPoints(
    np.concatenate(
        [
            np.linspace(X.min(), X.max(), NUM_INDUCING).reshape(-1, 1),
            np.random.randn(NUM_INDUCING, 1),
        ],
        axis=1
    )
)
gp_layer = gpflux.layers.GPLayer(
    kernel,
    inducing_variable,
    num_data=num_data,
    num_latent_gps=1,
    mean_function=gpflow.mean_functions.Zero(),
)

### Second GP layer

Final layer GP with Squared Exponential kernel

In [ ]:

kernel = gpflow.kernels.SquaredExponential()
inducing_variable = gpflow.inducing_variables.InducingPoints(
    np.random.randn(NUM_INDUCING, 1),
)
gp_layer2 = gpflux.layers.GPLayer(
    kernel,
    inducing_variable,
    num_data=num_data,
    num_latent_gps=1,
    mean_function=gpflow.mean_functions.Identity(),
)
gp_layer2.q_sqrt.assign(gp_layer.q_sqrt * 1e-5);

In [ ]:

likelihood_layer = gpflux.layers.LikelihoodLayer(gpflow.likelihoods.Gaussian(0.01))
dgp = gpflux.models.DeepGP([lv, gp_layer, gp_layer2], likelihood_layer)
gpflow.utilities.print_summary(dgp, fmt="notebook")

### Fit

We can now fit the model. Because of the `DirectlyParameterizedEncoder` it is important to set the batch size to the number of datapoints and turn off shuffle. This is so that we use the associated latent variable for each datapoint. If we would use an Amortized Encoder network this would not be necessary.

In [25]:
model = dgp.as_training_model()
model.compile(tf.optimizers.Adam(0.005))
history = model.fit({"inputs": X, "targets": Y}, epochs=int(10e3), verbose=0, batch_size=num_data, shuffle=False)


1/1 [==============================] - 0s 2ms/step - loss: 10.3615 - latent_variable_layer_local_kl: 9.5638 - gp_layer_2_prior_kl: 0.2760 - gp_layer_4_prior_kl: 0.6132
Epoch 294/7000
1/1 [==============================] - 0s 2ms/step - loss: 10.1598 - latent_variable_layer_local_kl: 9.5588 - gp_layer_2_prior_kl: 0.2765 - gp_layer_4_prior_kl: 0.6131
Epoch 295/7000
1/1 [==============================] - 0s 2ms/step - loss: 10.1892 - latent_variable_layer_local_kl: 9.5538 - gp_layer_2_prior_kl: 0.2770 - gp_layer_4_prior_kl: 0.6129
Epoch 296/7000
1/1 [==============================] - 0s 2ms/step - loss: 10.1331 - latent_variable_layer_local_kl: 9.5487 - gp_layer_2_prior_kl: 0.2775 - gp_layer_4_prior_kl: 0.6128
Epoch 297/7000
1/1 [==============================] - 0s 2ms/step - loss: 10.0920 - latent_variable_layer_local_kl: 9.5437 - gp_layer_2_prior_kl: 0.2780 - gp_layer_4_prior_kl: 0.6127
Epoch 298/7000
1/1 [==============================] - 0s 2ms/step - loss: 10.0540 - latent_variable_